In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.callbacks import EarlyStopping


from sklearn.model_selection import KFold, train_test_split

In [ ]:
Xtr = pd.read_pickle('./X_train1.pkl')
Xte = pd.read_pickle('./X_test1.pkl')
y = pd.read_pickle('../y_train.pkl')

In [ ]:
Xtr.shape

In [ ]:
# create model
model1 = Sequential()
model1.add(Dense(60, input_dim=Xtr.shape[1], kernel_initializer='normal', activation='relu'))
model1.add(Dropout(0.25))
model1.add(Dense(30, kernel_initializer='normal', activation='relu'))
model1.add(Dropout(0.25))
model1.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
# Compile model
model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


In [ ]:
Xtr.shape[1]

In [ ]:
# create model
model2 = Sequential()
model2.add(Dense(120,input_dim=Xtr.shape[1], kernel_initializer='normal', activation='relu'))
model2.add(Dropout(0.25))
model2.add(Dense(60, kernel_initializer='normal', activation='relu'))
model2.add(Dropout(0.25))
model2.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
# Compile model
model2.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
cb = [EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=1, mode='auto')]

model1.fit(x=Xtr.values, y=y, batch_size=32, epochs=15, callbacks=cb, verbose=2, validation_split=0.15)

In [ ]:
preds = model1.predict_proba(Xte.values)

In [ ]:
preds.ravel()

In [ ]:
pd.Series(preds.ravel()).to_csv('kerasmodel2.csv', index=False)

In [ ]:
def stackEnsemble(models, X, y, Xtest, splits, verbose):

    # assert correct data-types 
    assert type(models) == list
    assert type(splits) == int
    assert type(verbose) == bool

    # init variables
    kf = KFold(n_splits = splits)
    predsTR = {}
    predsTE = {}

    # iterate over all inserted models
    for n, model in enumerate(models):
        if verbose: print('Using model %d to make predictions..' % (n+1))

        # prepare split for predictions
        predsTR['model'+str(n+1)] = []
        for i, (train, test) in enumerate(kf.split(X)):
            if verbose: print('..on split %d' % (i+1))

            # fit on split and predict
            model.fit(X.iloc[train], y[train], batch_size=10, epochs=10)
            predsTR['model'+str(n+1)].append(list(model.predict_proba(X.iloc[test])[:, 1]))

        # predict on testset
        predsTE['model'+str(n+1)] = list(model.predict_proba(Xtest)[:, 1])
    
    # combine trainset predictions in dataframe, join with trainset
    meta_feats = pd.DataFrame(columns = [col for col in predsTR.keys()])
    for model in predsTR.keys():
        meta_feats[model] = np.array([item for lst in predsTR[model] for item in lst])
    X = pd.concat([X, meta_feats], axis=1)

    # combine testset predictions in dataframe, join with testset
    meta_feats = pd.DataFrame(columns = [col for col in predsTE.keys()])
    for model in predsTE.keys():
        meta_feats[model] = np.array(predsTE[model])
    Xtest = pd.concat([Xtest, meta_feats], axis=1)

    # return trainset and testset with metafeatures
    return X, Xtest

In [ ]:
models = [model1, model2]
X_train_stack, X_test_stack = stackEnsemble(models, Xtr, y, Xte, 4, True)